[View in Colaboratory](https://colab.research.google.com/github/Angordil/Analisis-de-registros-de-EEG-mediante-redes-neuronales/blob/master/Dense.ipynb)

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip install -q keras

In [0]:
import numpy as np
import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers import Conv2D
from tensorflow.python.keras.applications import imagenet_utils

## Load training sets for one subject
X_training = np.load('drive/TFG/Trial only with subject 1/Datos/X_training.npy')
y_training1hot = np.load('drive/TFG/Trial only with subject 1/Datos/y_training1hot.npy')

## Load training and validaton sets for all subjects
# X_training = np.load('drive/TFG/training/X3_train.npy')
# y_training1hot = np.load('drive/TFG/training/y_training1hot.npy')
# X_validation = np.load('drive/TFG/validation/X3_val.npy')
# y_validation1hot = np.load('drive/TFG/validation/y_validation1hot.npy')

print X_training.shape
print y_training1hot.shape



In [0]:
num_classes = 6

model = Sequential()

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Train model for one subject
history = model.fit(X_training, y_training1hot, epochs = 24, validation_split = 0.1, shuffle = True)

# Train model for all subjects
# history = model.fit(X_training, y_training1hot, epochs = 24, validation_data = [X_validation, y_validation1hot], shuffle = True)

In [0]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])

plt.plot(history.history['val_acc'])

plt.title('Accuracy over time')
plt.ylabel('accuracy')
plt.xlabel('epoch')

plt.legend(['training', 'validation'], loc = 'upper left')
plt.show

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('Cross Entropy Loss over Time')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc = 'upper left')
plt.show

In [0]:
# ### Model Quality Check: Confusion Matrix

import itertools
from sklearn.metrics import confusion_matrix

# Load validation set for one subject
y_validation = np.load('drive/TFG/Trial only with subject 1/Datos/y_training.npy')
y_validation = y_validation[-519:]
y_validation_predictions = modelk.predict(X_training[-519:], verbose = 1)

# Load validation set for all subjects
# y_validation = np.load('drive/TFG/validation/y_val.npy')
# y_validation_predictions = modelk.predict(X_validation, verbose = 1)

def plot_confusion_matrix(cm, classes, normalize = False, title = 'Confusion matrix', cmap = plt.cm.Blues):
  """
  This function prints and plots the confusion matrix.
  Normalization can be applied by setting 'normalize = True'
  """
  plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation = 45)
  plt.yticks(tick_marks, classes)
  
  if normalize:
    cm = cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
    print('Normalized confusion matrix')
  else:
    print('Confusion matrix, without normalization')
    
  print(cm)
  
  thresh = cm.max()/2.
  for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment = 'center', color = 'white' if cm[i, j] > thresh else 'black')
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
cnf_matrix = confusion_matrix(y_validation - 1, np.argmax(y_validation_predictions, axis = 1))

plt.figure()
class_names = ['Human Body', 'Human Face', 'Animal Body', 'Animal Face', 'Fruit Vegetable', 'Inanimate Object']
plot_confusion_matrix(cnf_matrix, classes = class_names)